In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from hyperopt import fmin, hp,tpe,Trials,STATUS_OK
import pickle
import time

In [2]:
all_data = pd.read_csv('all_data.csv')

In [3]:
to_drop_cols = ['target_item_avg', 'target_item', 'target', 'target_shop_avg', 'target_shop', 'date_block_num']

X_train = all_data[all_data['date_block_num']<33]
Y_train = X_train['target']
X_train = X_train.drop(to_drop_cols,axis=1)
X_val = all_data[all_data['date_block_num']==33]
Y_val = X_val['target']
X_val = X_val.drop(to_drop_cols,axis=1)
X_test = all_data[all_data['date_block_num']==34].drop(to_drop_cols,axis=1)

In [4]:
Y_train.clip(0,40,inplace=True)
Y_val.clip(0,40,inplace=True)

/home/chiayi/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(result)


In [5]:
def xgb_score(params):
    watchlist = [(xgb.DMatrix(X_train, Y_train), 'train'), (xgb.DMatrix(X_val, Y_val), 'valid')]
    model = xgb.train(params, xgb.DMatrix(X_train, Y_train), 3500,  watchlist, maximize=False, verbose_eval=50, early_stopping_rounds=50)
    pred = xgb.predict(X_val, ntree_limit=model.best_ntree_limit)
    loss = metrics.r2_score(Y_val, pred)
    return {'loss': loss, 'status': STATUS_OK}

In [9]:
def xgb_hyperopt(evals, cores, trials, optimizer=tpe.suggest, random_state=0):
    space = {
        'eta': hp.quniform('eta',0.01,0.5,0.01),
        'max_depth': hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight',0,100,1),
        'subsample':hp.quniform('subsample',0.1,1.0,0.1),
        'gamma': hp.quniform('gamma',0.0,30,0.5),
        'colsample_bytree': hp.quniform('colsample_bytree',0.1,1.0,0.1),
        'objective':'reg:linear',
        'nthread':cores,
        'silent':1,
        'num_round':2500,
        'seed':2441,
        'early_stopping_rounds':100
    }

    trials = Trials()
    best = fmin(xgb_score,space,algo=tpe.suggest,max_evals=evals,trials=trials)
    return best

In [10]:
trials = Trials()
start = time.time()
best = xgb_hyperopt(evals = 1000, optimizer=tpe.suggest, cores = 2,trials = trials)
print ('Best parameters are')
print (best)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(n,end - start))

MemoryError: 